In [ ]:
import pandas as pd
import os
import datetime
# Variables
input_dir = 'path_to_input_csv_files'  # Replace with your input CSV files directory
output_dir = 'path_to_output_csv_files'  # Replace with your output CSV files directory
city = 'NYC'  # or 'CHI'

# Query 3
n = 2  # Integer >= 2
from_date_str = '01/01/2021'  # Format 'DD/MM/YYYY'
to_date_str = '31/12/2021'  # Format 'DD/MM/YYYY'


# Query 4
n = 3  # Integer >= 1
agency_param = 'DEPARTMENT_OF_TRANSPORTATION'  # Use underscores "_" instead of spaces

In [ ]:
# Query 1: Total de multas por infracción y agencia

# Read data based on the city
if city == 'NYC':
    tickets_file = os.path.join(input_dir, 'ticketsNYC.csv')
    infractions_file = os.path.join(input_dir, 'infractionsNYC.csv')
    agencies_file = os.path.join(input_dir, 'agenciesNYC.csv')
    # Read the data
    tickets = pd.read_csv(tickets_file, sep=';')
    infractions = pd.read_csv(infractions_file, sep=';')
    agencies = pd.read_csv(agencies_file, sep=';')
    # Rename columns for consistency
    tickets.rename(columns={
        'Plate': 'Plate',
        'Infraction ID': 'Infraction ID',
        'Fine Amount': 'Fine Amount',
        'Issuing Agency': 'Agency',
        'Issue Date': 'Issue Date',
        'County Name': 'County'
    }, inplace=True)
    infractions.rename(columns={
        'Infraction ID': 'Infraction ID',
        'Definition': 'Infraction'
    }, inplace=True)
    agencies.rename(columns={
        'Issuing Agency': 'Agency'
    }, inplace=True)
else:
    # CHI
    tickets_file = os.path.join(input_dir, 'ticketsCHI.csv')
    infractions_file = os.path.join(input_dir, 'infractionsCHI.csv')
    agencies_file = os.path.join(input_dir, 'agenciesCHI.csv')
    # Read the data
    tickets = pd.read_csv(tickets_file, sep=';')
    infractions = pd.read_csv(infractions_file, sep=';')
    agencies = pd.read_csv(agencies_file, sep=';')
    # Rename columns for consistency
    tickets.rename(columns={
        'issue_date': 'Issue Date',
        'community_area_name': 'County',
        'unit_description': 'Agency',
        'license_plate_number': 'Plate',
        'violation_code': 'Infraction ID',
        'fine_amount': 'Fine Amount'
    }, inplace=True)
    infractions.rename(columns={
        'violation_code': 'Infraction ID',
        'violation_description': 'Infraction'
    }, inplace=True)
    agencies.rename(columns={
        'agency_name': 'Agency'
    }, inplace=True)

# Filter tickets with valid infractions and agencies
valid_infractions = infractions['Infraction ID'].unique()
valid_agencies = agencies['Agency'].unique()
tickets = tickets[tickets['Infraction ID'].isin(valid_infractions) & tickets['Agency'].isin(valid_agencies)]

# Merge tickets with infractions to get infraction names
tickets = tickets.merge(infractions, on='Infraction ID', how='left')

# Group by Infraction and Agency, count tickets
grouped = tickets.groupby(['Infraction', 'Agency']).size().reset_index(name='Tickets')

# Sort descending by Tickets, then alphabetical by Infraction and Agency
grouped.sort_values(by=['Tickets', 'Infraction', 'Agency'], ascending=[False, True, True], inplace=True)

# Output the result
output_file = os.path.join(output_dir, 'query1.csv')
grouped[['Infraction', 'Agency', 'Tickets']].to_csv(output_file, sep=';', index=False)


In [ ]:
# Read data based on the city
if city == 'NYC':
    tickets_file = os.path.join(input_dir, 'ticketsNYC.csv')
    agencies_file = os.path.join(input_dir, 'agenciesNYC.csv')
    tickets = pd.read_csv(tickets_file, sep=';')
    agencies = pd.read_csv(agencies_file, sep=';')
    tickets.rename(columns={
        'Plate': 'Plate',
        'Infraction ID': 'Infraction ID',
        'Fine Amount': 'Fine Amount',
        'Issuing Agency': 'Agency',
        'Issue Date': 'Issue Date',
        'County Name': 'County'
    }, inplace=True)
    agencies.rename(columns={'Issuing Agency': 'Agency'}, inplace=True)
else:
    tickets_file = os.path.join(input_dir, 'ticketsCHI.csv')
    agencies_file = os.path.join(input_dir, 'agenciesCHI.csv')
    tickets = pd.read_csv(tickets_file, sep=';')
    agencies = pd.read_csv(agencies_file, sep=';')
    tickets.rename(columns={
        'issue_date': 'Issue Date',
        'community_area_name': 'County',
        'unit_description': 'Agency',
        'license_plate_number': 'Plate',
        'violation_code': 'Infraction ID',
        'fine_amount': 'Fine Amount'
    }, inplace=True)
    agencies.rename(columns={'agency_name': 'Agency'}, inplace=True)

# Convert 'Issue Date' to datetime
if city == 'NYC':
    tickets['Issue Date'] = pd.to_datetime(tickets['Issue Date'], format='%Y-%m-%d')
else:
    tickets['Issue Date'] = pd.to_datetime(tickets['Issue Date'], format='%Y-%m-%d %H:%M:%S')

# Filter tickets with valid agencies
valid_agencies = agencies['Agency'].unique()
tickets = tickets[tickets['Agency'].isin(valid_agencies)]

# Extract Year and Month
tickets['Year'] = tickets['Issue Date'].dt.year
tickets['Month'] = tickets['Issue Date'].dt.month

# Ensure 'Fine Amount' is numeric
tickets['Fine Amount'] = pd.to_numeric(tickets['Fine Amount'], errors='coerce').fillna(0)

# Group by Agency, Year, Month, sum 'Fine Amount'
monthly_revenue = tickets.groupby(['Agency', 'Year', 'Month'])['Fine Amount'].sum().reset_index(name='Monthly Revenue')

# Sort data for cumulative sum
monthly_revenue.sort_values(by=['Agency', 'Year', 'Month'], inplace=True)

# Calculate YTD revenue per agency per year
def compute_ytd(group):
    group['YTD'] = group['Monthly Revenue'].cumsum()
    return group

ytd_revenue = monthly_revenue.groupby(['Agency', 'Year']).apply(compute_ytd)

# Remove months with zero YTD revenue
ytd_revenue = ytd_revenue[ytd_revenue['YTD'] > 0]

# Prepare the output
output = ytd_revenue[['Agency', 'Year', 'Month', 'YTD']]

# Output the result
output_file = os.path.join(output_dir, 'query2.csv')
output.to_csv(output_file, sep=';', index=False)


In [ ]:
# Query 3: Porcentaje de patentes reincidentes por barrio en el rango [from, to]

# Read data based on the city
if city == 'NYC':
    tickets_file = os.path.join(input_dir, 'ticketsNYC.csv')
    tickets = pd.read_csv(tickets_file, sep=';')
    tickets.rename(columns={
        'Plate': 'Plate',
        'Infraction ID': 'Infraction ID',
        'Fine Amount': 'Fine Amount',
        'Issuing Agency': 'Agency',
        'Issue Date': 'Issue Date',
        'County Name': 'County'
    }, inplace=True)
else:
    tickets_file = os.path.join(input_dir, 'ticketsCHI.csv')
    tickets = pd.read_csv(tickets_file, sep=';')
    tickets.rename(columns={
        'issue_date': 'Issue Date',
        'community_area_name': 'County',
        'unit_description': 'Agency',
        'license_plate_number': 'Plate',
        'violation_code': 'Infraction ID',
        'fine_amount': 'Fine Amount'
    }, inplace=True)

# Convert 'Issue Date' to datetime
date_format = '%d/%m/%Y'
from_date = datetime.datetime.strptime(from_date_str, date_format)
to_date = datetime.datetime.strptime(to_date_str, date_format)

if city == 'NYC':
    tickets['Issue Date'] = pd.to_datetime(tickets['Issue Date'], format='%Y-%m-%d')
else:
    tickets['Issue Date'] = pd.to_datetime(tickets['Issue Date'], format='%Y-%m-%d %H:%M:%S')

# Filter tickets in date range [from, to]
tickets = tickets[(tickets['Issue Date'] >= from_date) & (tickets['Issue Date'] <= to_date)]

# Total unique plates per county
total_plates = tickets.groupby('County')['Plate'].nunique().reset_index(name='Total Plates')

# Identify repeat offenders
grouped = tickets.groupby(['County', 'Plate', 'Infraction ID']).size().reset_index(name='Count')
repeat_offenders = grouped[grouped['Count'] >= n]
repeat_offenders = repeat_offenders.groupby('County')['Plate'].nunique().reset_index(name='Repeat Offender Plates')

# Merge and calculate percentage
result = total_plates.merge(repeat_offenders, on='County', how='left').fillna(0)
result['Percentage'] = (result['Repeat Offender Plates'] / result['Total Plates'] * 100).apply(lambda x: float(int(x * 100)) / 100.0)

# Sort descending by Percentage, then County
result.sort_values(by=['Percentage', 'County'], ascending=[False, True], inplace=True)

# Output the result
output = result[['County', 'Percentage']]
output_file = os.path.join(output_dir, 'query3.csv')
output.to_csv(output_file, sep=';', index=False)


In [ ]:
# Query 4: Top N infracciones con mayor diferencia entre máximos y mínimos montos para una agencia

# Convert agency_param to agency name
agency_name = agency_param.replace('_', ' ')

# Read data based on the city
if city == 'NYC':
    tickets_file = os.path.join(input_dir, 'ticketsNYC.csv')
    infractions_file = os.path.join(input_dir, 'infractionsNYC.csv')
    tickets = pd.read_csv(tickets_file, sep=';')
    infractions = pd.read_csv(infractions_file, sep=';')
    tickets.rename(columns={
        'Plate': 'Plate',
        'Infraction ID': 'Infraction ID',
        'Fine Amount': 'Fine Amount',
        'Issuing Agency': 'Agency',
        'Issue Date': 'Issue Date',
        'County Name': 'County'
    }, inplace=True)
    infractions.rename(columns={'Infraction ID': 'Infraction ID', 'Definition': 'Infraction'}, inplace=True)
else:
    tickets_file = os.path.join(input_dir, 'ticketsCHI.csv')
    infractions_file = os.path.join(input_dir, 'infractionsCHI.csv')
    tickets = pd.read_csv(tickets_file, sep=';')
    infractions = pd.read_csv(infractions_file, sep=';')
    tickets.rename(columns={
        'issue_date': 'Issue Date',
        'community_area_name': 'County',
        'unit_description': 'Agency',
        'license_plate_number': 'Plate',
        'violation_code': 'Infraction ID',
        'fine_amount': 'Fine Amount'
    }, inplace=True)
    infractions.rename(columns={'violation_code': 'Infraction ID', 'violation_description': 'Infraction'}, inplace=True)

# Filter tickets for the specified agency
tickets = tickets[tickets['Agency'] == agency_name]

# Ensure 'Fine Amount' is numeric
tickets['Fine Amount'] = pd.to_numeric(tickets['Fine Amount'], errors='coerce').fillna(0)

# Filter tickets with infractions present in infractions.csv
valid_infractions = infractions['Infraction ID'].unique()
tickets = tickets[tickets['Infraction ID'].isin(valid_infractions)]

# Merge tickets with infractions to get infraction names
tickets = tickets.merge(infractions, on='Infraction ID', how='left')

# Calculate min, max, and difference for each infraction
grouped = tickets.groupby('Infraction')['Fine Amount'].agg(['min', 'max']).reset_index()
grouped['Diff'] = grouped['max'] - grouped['min']

# Sort and select top N infractions
grouped.sort_values(by=['Diff', 'Infraction'], ascending=[False, True], inplace=True)
output = grouped.head(n)

# Prepare and output the result
output.rename(columns={'min': 'Min', 'max': 'Max'}, inplace=True)
output_file = os.path.join(output_dir, 'query4.csv')
output[['Infraction', 'Min', 'Max', 'Diff']].to_csv(output_file, sep=';', index=False)
